In [4]:
!pip uninstall -y numpy
!pip install numpy==1.25.2
!pip install pandas==2.0.3
!pip install scipy==1.13.1
!pip install --upgrade gensim tensorflow #Upgrade gensim and tensorflow
!pip install scikit-learn nltk imblearn

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 65.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
blosc2 3.2.0 requires numpy>=1.26, but you have numpy 1.25.2 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.25.2 which is incompatible.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 45.3 MB/s eta 0:00:00
^C
^C
^C
^C


In [2]:
!pip install gensim


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 651.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 9.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1


In [3]:
!pip install emoji


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 18.2 MB/s eta 0:00:00


In [13]:
!pip install PySastrawi

In [2]:
import pandas as pd
import re
import emoji
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from gensim.models import Word2Vec
from Sastrawi.Stemmer import StemmerFactory
from imblearn.over_sampling import SMOTE
import joblib
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# Inisialisasi stemmer Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory # Correctly import StemmerFactory class
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Load kamus slang eksternal (misalnya dari nasalsabila/kamus-alay)
slang_df = pd.read_csv('https://raw.githubusercontent.com/nasalsabila/kamus-alay/refs/heads/master/colloquial-indonesian-lexicon.csv')  # Unduh dari GitHub atau buat sendiri
slang_dict = dict(zip(slang_df['slang'], slang_df['formal']))

In [4]:
df = pd.read_csv('https://raw.githubusercontent.com/miezlearning/Coding-Camp-2025/refs/heads/master/Belajar%20Pengembangan%20Machine%20Learning/data/hasil_review_genshin_impact.csv')
print("Jumlah data:", len(df))
print(df.head())

Jumlah data: 15000
                               reviewId         userName  \
0  f70e716a-49a4-4c1a-a0d2-83e076f6bec0  Pengguna Google   
1  020413b0-06ce-4bf1-81c1-3cbe97685507  Pengguna Google   
2  2cc3ddee-9df3-4342-921b-5f436aa9cdf1  Pengguna Google   
3  205af55a-bff1-49f4-8cb3-39123215e0c1  Pengguna Google   
4  427ce1da-c883-4cfe-af5d-07dc71d5aa1f  Pengguna Google   

                                           userImage  \
0  https://play-lh.googleusercontent.com/EGemoI2N...   
1  https://play-lh.googleusercontent.com/EGemoI2N...   
2  https://play-lh.googleusercontent.com/EGemoI2N...   
3  https://play-lh.googleusercontent.com/EGemoI2N...   
4  https://play-lh.googleusercontent.com/EGemoI2N...   

                                             content  score  thumbsUpCount  \
0                                menyenangkan sekali      5              0   
1                                  Adventure Terbaik      5              0   
2  yahh, meskipun handphone saya kentang masih bi

In [5]:
# Pelabelan sentimen
def label_sentiment(score):
    if score <= 2: return 'negatif'
    elif score == 3: return 'netral'
    else: return 'positif'

df['sentiment'] = df['score'].apply(label_sentiment)
print("Distribusi awal:\n", df['sentiment'].value_counts().to_string())

# Stopwords tambahan dari NLTK
stop_words = set(stopwords.words('indonesian')) | {'dan', 'yang', 'di', 'ke', 'nya', 'ini', 'itu'}

# Fungsi pembersihan teks yang lebih baik
def clean_text(text):
    text = str(text).lower()
    text = emoji.replace_emoji(text, replace='')
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    words = text.split()
    # Pertahankan kata pendek yang relevan
    text = ' '.join(slang_dict.get(word, word) for word in words if word not in stop_words or word in ['oke', 'bagus', 'top'])
    return stemmer.stem(text)

df['cleaned_content'] = df['content'].apply(clean_text)

# Oversampling dengan SMOTE setelah TF-IDF
tfidf = TfidfVectorizer(max_features=10000, stop_words=list(stop_words), ngram_range=(1, 2))
X_tfidf = tfidf.fit_transform(df['cleaned_content']).toarray()
y = pd.get_dummies(df['sentiment']).values
smote = SMOTE(random_state=42)
X_tfidf_smote, y_smote = smote.fit_resample(X_tfidf, np.argmax(y, axis=1))
y_smote = pd.get_dummies(y_smote).values
df_balanced = pd.DataFrame({'cleaned_content': [' '.join(doc) for doc in tfidf.inverse_transform(X_tfidf_smote)], 'sentiment': np.argmax(y_smote, axis=1)})
df_balanced['sentiment'] = df_balanced['sentiment'].map({0: 'negatif', 1: 'netral', 2: 'positif'})
print(f"Jumlah data setelah SMOTE: {len(df_balanced)}")
print("Distribusi setelah SMOTE:\n", df_balanced['sentiment'].value_counts().to_string())

# Fungsi evaluasi
def evaluate_model(y_true, y_pred, set_name=""):
    accuracy = accuracy_score(y_true, y_pred)
    print(f"\nAkurasi {set_name}: {accuracy * 100:.2f}%")
    print(classification_report(y_true, y_pred, target_names=['negatif', 'netral', 'positif']))
    return accuracy

Distribusi awal:
 sentiment
positif    10101
negatif     4001
netral       898


/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['baiknya', 'berkali', 'kali', 'kurangnya', 'mata', 'olah', 'sekurang', 'setidak', 'tama', 'tidaknya'] not in stop_words.
  warnings.warn(


Jumlah data setelah SMOTE: 30303
Distribusi setelah SMOTE:
 sentiment
positif    10101
negatif    10101
netral     10101


In [6]:
# --- Skema 1: Dense + TF-IDF (80/20) ---
print("\n=== Skema 1: Dense + TF-IDF (80/20) ===")
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_tfidf_smote, y_smote, test_size=0.2, random_state=42)

model1 = Sequential([
    Input(shape=(10000,)),
    Dense(512, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.4),
    Dense(256, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.4),
    Dense(3, activation='softmax')
])
model1.compile(optimizer=Adam(learning_rate=0.0005), loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.00001)
model1.fit(X_train1, y_train1, epochs=30, batch_size=64, validation_split=0.1, callbacks=[early_stopping, lr_scheduler], verbose=1)

y_pred_train1 = np.argmax(model1.predict(X_train1), axis=1)
y_pred_test1 = np.argmax(model1.predict(X_test1), axis=1)
y_train1_cat = np.argmax(y_train1, axis=1)
y_test1_cat = np.argmax(y_test1, axis=1)
train_acc1 = evaluate_model(y_train1_cat, y_pred_train1, "Training")
test_acc1 = evaluate_model(y_test1_cat, y_pred_test1, "Testing")

# --- Skema 2: LSTM + Word2Vec (80/20) ---
print("\n=== Skema 2: LSTM + Word2Vec (80/20) ===")
sentences = [text.split() for text in df_balanced['cleaned_content']]
w2v_model = Word2Vec(sentences, vector_size=200, window=5, min_count=1, workers=4, epochs=20)

tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(df_balanced['cleaned_content'])
X_seq = tokenizer.texts_to_sequences(df_balanced['cleaned_content'])
max_len = 100  # Dikurangi untuk teks pendek
X_pad = pad_sequences(X_seq, maxlen=max_len)

embedding_matrix = np.zeros((10000, 200))
for word, i in tokenizer.word_index.items():
    if i < 10000 and word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]

X_train2, X_test2, y_train2, y_test2 = train_test_split(X_pad, y_smote, test_size=0.2, random_state=42)

model2 = Sequential([
    Embedding(10000, 200, weights=[embedding_matrix], input_length=max_len, trainable=True),
    Bidirectional(LSTM(256, return_sequences=True, kernel_regularizer=l2(0.005))),
    LSTM(128),
    Dropout(0.5),
    Dense(128, activation='relu', kernel_regularizer=l2(0.005)),
    Dropout(0.5),
    Dense(3, activation='softmax')
])
model2.compile(optimizer=Adam(learning_rate=0.0005), loss='categorical_crossentropy', metrics=['accuracy'])

model2.fit(X_train2, y_train2, epochs=30, batch_size=64, validation_split=0.1, callbacks=[early_stopping, lr_scheduler], verbose=1)

y_pred_train2 = np.argmax(model2.predict(X_train2), axis=1)
y_pred_test2 = np.argmax(model2.predict(X_test2), axis=1)
y_train2_cat = np.argmax(y_train2, axis=1)
y_test2_cat = np.argmax(y_test2, axis=1)
train_acc2 = evaluate_model(y_train2_cat, y_pred_train2, "Training")
test_acc2 = evaluate_model(y_test2_cat, y_pred_test2, "Testing")

# --- Skema 3: Dense + TF-IDF (70/30) ---
print("\n=== Skema 3: Dense + TF-IDF (70/30) ===")
X_train3, X_test3, y_train3, y_test3 = train_test_split(X_tfidf_smote, y_smote, test_size=0.3, random_state=42)

model3 = Sequential([
    Input(shape=(10000,)),
    Dense(512, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.4),
    Dense(256, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.4),
    Dense(3, activation='softmax')
])
model3.compile(optimizer=Adam(learning_rate=0.0005), loss='categorical_crossentropy', metrics=['accuracy'])

model3.fit(X_train3, y_train3, epochs=30, batch_size=64, validation_split=0.1, callbacks=[early_stopping, lr_scheduler], verbose=1)

y_pred_train3 = np.argmax(model3.predict(X_train3), axis=1)
y_pred_test3 = np.argmax(model3.predict(X_test3), axis=1)
y_train3_cat = np.argmax(y_train3, axis=1)
y_test3_cat = np.argmax(y_test3, axis=1)
train_acc3 = evaluate_model(y_train3_cat, y_pred_train3, "Training")
test_acc3 = evaluate_model(y_test3_cat, y_pred_test3, "Testing")


=== Skema 1: Dense + TF-IDF (80/20) ===
Epoch 1/30
341/341 ━━━━━━━━━━━━━━━━━━━━ 16s 23ms/step - accuracy: 0.5590 - loss: 1.3069 - val_accuracy: 0.7794 - val_loss: 0.8123 - learning_rate: 5.0000e-04
Epoch 2/30
341/341 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8013 - loss: 0.7482 - val_accuracy: 0.8247 - val_loss: 0.7553 - learning_rate: 5.0000e-04
Epoch 3/30
341/341 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8376 - loss: 0.6946 - val_accuracy: 0.8231 - val_loss: 0.7185 - learning_rate: 5.0000e-04
Epoch 4/30
341/341 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.8667 - loss: 0.6357 - val_accuracy: 0.8487 - val_loss: 0.7048 - learning_rate: 5.0000e-04
Epoch 5/30
341/341 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8722 - loss: 0.6193 - val_accuracy: 0.8375 - val_loss: 0.6947 - learning_rate: 5.0000e-04
Epoch 6/30
341/341 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8865 - loss: 0.5860 - val_accuracy: 0.8219 - val_loss: 0.6815 - learning_rate: 5.0000e-04
Epoch 7/30
341/341 ━━

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


341/341 ━━━━━━━━━━━━━━━━━━━━ 20s 36ms/step - accuracy: 0.5473 - loss: 2.8946 - val_accuracy: 0.6095 - val_loss: 1.1684 - learning_rate: 5.0000e-04
Epoch 2/30
341/341 ━━━━━━━━━━━━━━━━━━━━ 16s 34ms/step - accuracy: 0.6494 - loss: 1.0662 - val_accuracy: 0.6685 - val_loss: 0.9290 - learning_rate: 5.0000e-04
Epoch 3/30
341/341 ━━━━━━━━━━━━━━━━━━━━ 20s 34ms/step - accuracy: 0.7014 - loss: 0.8549 - val_accuracy: 0.7357 - val_loss: 0.7768 - learning_rate: 5.0000e-04
Epoch 4/30
341/341 ━━━━━━━━━━━━━━━━━━━━ 11s 33ms/step - accuracy: 0.7555 - loss: 0.7420 - val_accuracy: 0.7332 - val_loss: 0.7453 - learning_rate: 5.0000e-04
Epoch 5/30
341/341 ━━━━━━━━━━━━━━━━━━━━ 21s 34ms/step - accuracy: 0.7914 - loss: 0.6601 - val_accuracy: 0.7984 - val_loss: 0.6427 - learning_rate: 5.0000e-04
Epoch 6/30
341/341 ━━━━━━━━━━━━━━━━━━━━ 20s 33ms/step - accuracy: 0.8218 - loss: 0.5909 - val_accuracy: 0.8120 - val_loss: 0.6062 - learning_rate: 5.0000e-04
Epoch 7/30
341/341 ━━━━━━━━━━━━━━━━━━━━ 11s 33ms/step - accurac

In [8]:
# Fungsi inference
def predict_sentiment(text, model, vectorizer, is_word2vec=False, tokenizer=None, max_len=100):
    cleaned_text = clean_text(text)
    if is_word2vec:
        seq = tokenizer.texts_to_sequences([cleaned_text])
        padded = pad_sequences(seq, maxlen=max_len)
        pred = model.predict(padded)
    else:
        tfidf_vec = vectorizer.transform([cleaned_text]).toarray()
        pred = model.predict(tfidf_vec)
    sentiment = np.argmax(pred, axis=1)[0]
    return ['negatif', 'netral', 'positif'][sentiment]

# Contoh inference
sample_text = "game nya bagus banget mantap"
print("\nContoh Inference:")
print(f"Skema 1 (Dense+TF-IDF): {predict_sentiment(sample_text, model1, tfidf)}")
print(f"Skema 2 (LSTM+Word2Vec): {predict_sentiment(sample_text, model2, None, True, tokenizer, max_len)}")
print(f"Skema 3 (Dense+TF-IDF): {predict_sentiment(sample_text, model3, tfidf)}")


Contoh Inference:
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 823ms/step
Skema 1 (Dense+TF-IDF): positif
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 285ms/step
Skema 2 (LSTM+Word2Vec): positif
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 413ms/step
Skema 3 (Dense+TF-IDF): positif


In [9]:
print(df_balanced['sentiment'].value_counts().to_string())


sentiment
positif    10101
negatif    10101
netral     10101
